In [ ]:
# adaptation of CAN_P.m by Carl Tape
# coding by: Yongki Aiman, Tomy Gunawan, Angel Ling

# analyzing the Sumatra earthquake P wave

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from matplotlib.mlab import detrend
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

from lib_seis import sumatra_event
from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 0.5

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for item in event:
    print(f'{item} = {event[item]}')
print()    

In [ ]:
# getting waveforms (all three components)

client = Client("IRIS")

starttime  = event['origin_time'] - ( 1 * 60 * 60 )           # 1 hour before origin time
endtime    = event['origin_time'] + ( 3 * 60 * 60)            # 3 hours after origin time

st = client.get_waveforms("G", "CAN", "*", "B*", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plotting all three components of the seismogram

can   = st.copy()

can_z = can.select(component="Z")
can_n = can.select(component="N")
can_e = can.select(component="E")

t     = can[0].times()

tr_z  = can_z[0]
tr_n  = can_n[0]
tr_e  = can_e[0]

title_z = f'{tr_z.id} − starting {tr_z.stats["starttime"]}'
title_n = f'{tr_n.id} − starting {tr_n.stats["starttime"]}'
title_e = f'{tr_e.id} − starting {tr_e.stats["starttime"]}'

fig, (ax1,ax2,ax3) = plt.subplots(3,1)
fig.set_size_inches(10,16)

ax1.plot(t, tr_z.data, color='b')
ax1.set(xlabel='seconds', ylabel='counts', title=title_z)

ax2.plot(t, tr_n.data, color='b')
ax2.set(xlabel='seconds', ylabel='counts', title=title_n)

ax3.plot(t, tr_e.data, color='b')
ax3.set(xlabel='seconds', ylabel='counts', title=title_e)

fig.show()

In [ ]:
# Z component only

plt.plot(t, tr_z.data, color='b')
plt.title(title_z)
plt.xlabel('seconds')
plt.ylabel('counts')

In [ ]:
# extracting a subset time series of the Z component containing the P wave

st1 = st.select(component="Z").copy()

t1 = starttime + 4000                # change these to 'pick' the P wave
t2 = starttime + 6000                # change these to 'pick' the P wave

st1.trim(t1,t2)
print(st1)

t  = st1[0].times()
tr = st1[0]

title = f'{tr.id} − starting {tr.stats["starttime"]}'

plt.plot(t, tr.data, color='b')
plt.title(title)
plt.xlabel('seconds')
plt.ylabel('counts')
plt.show()

In [ ]:
# do spectral analysis here

sampling_frequency = st1[0].stats.sampling_rate
Nyquist_frequency  = sampling_frequency / 2